# Computer Assignment 2: Connect-4 MinMax
In this assignment the game logic is already implemented. The task is to implement MinMax algorithm with/without alpha-beat pruning for [connect-4 game](https://en.wikipedia.org/wiki/Connect_Four).


> Connect Four (also known as Four Up, Plot Four, Find Four, Captain's Mistress, Four in a Row, Drop Four, and Gravitrips in the Soviet Union) is a two-player connection board game, in which the players choose a color and then take turns dropping colored tokens into a seven-column, six-row vertically suspended grid. The pieces fall straight down, occupying the lowest available space within the column. The objective of the game is to be the first to form a horizontal, vertical, or diagonal line of four of one's own tokens. Connect Four is a solved game. The first player can always win by playing the right moves.[(Source)](https://en.wikipedia.org/wiki/Connect_Four)

Author: **Danial Saeedi** (Student ID: 810198571)


# Heuristic Function

The heuristic adds a point to a player for each empty slot that could grant a player victory. The `calculate_score` calculates the heuristic value for each game state. The score that `calculate_score` returns is the cumulative score per coordination/point.

`point_score_calculator` : Each coordinate evaluates if a possible win can be found vertically, horizontally or in both diagonals.

**Note:** `calculate_score` returns $+\infty$ if the player(YOU) wins and returns $-\infty$ if the computer(CPU) wins.

# Import dependencies

In [2]:
from random import random
import copy
import numpy as np
import time

# MinMax without pruning

In [3]:
class MinMaxHandler:
    def __init__(self,cols = 7, rows = 6, search_depth = 4, CPU = -1, HUMAN_PLAYER = 1):
        self.cols = cols
        self.rows = rows
        self.search_depth = search_depth
        self.CPU = CPU
        self.HUMAN_PLAYER = HUMAN_PLAYER
    
    def calculate_number_of_possible_moves(self,game_state):
        possible_moves = self.cols
        for i in range(0, self.cols):
            if game_state[0][i] != 0:
                possible_moves -= 1
        return possible_moves

    def minimax(self,game_state, depth, player, opponent):
        possible_moves = self.calculate_number_of_possible_moves(game_state)
        
        # Evaluate Socre if there's possible moves or the depth is 0 and also return None for best_column
        if depth == 0 or possible_moves == 0:
            score = self.calculate_score(game_state, player, opponent)
            return None, score

        highest_score = None
        optimal_column = None

        for i in range(0, self.cols):
            # Skip this col if col is full
            if game_state[0][i] != 0:
                continue

            currentMove = [0, i]

            for j in range(0, self.rows - 1):
                if game_state[j + 1][i] != 0:
                    game_state[j][i] = player
                    currentMove[0] = j
                    break
                elif j == self.rows - 2:
                    game_state[j+1][i] = player
                    currentMove[0] = j+1

            # Recursive minimax call, with reduced depth
            move, score = self.minimax(game_state, depth - 1, opponent, player)

            game_state[currentMove[0]][currentMove[1]] = 0

            if player == self.CPU:
                if highest_score == None or score > highest_score:
                    highest_score = score
                    optimal_column = currentMove
            else:
                if highest_score == None or score < highest_score:
                    highest_score = score
                    optimal_column = currentMove

        return optimal_column, highest_score

    """
    This method calculates the score of each game state(board). 
    Return +inf if the you win and return -inf if CPU wins.
    """
    def calculate_score(self,game_state, player, opponent):
        # Return infinity if a self.YOU has won in the given board
        total_score = self.get_game_status(game_state)

        if total_score == player:
            return float("inf")
        elif total_score == opponent:
            return float("-inf")
        else:
            total_score = 0

        for i in range(0, self.rows):
            for j in range(0, self.cols):
                if game_state[i][j] == 0:
                    total_score += self.point_score_calculator(game_state, i, j, player, opponent)

        return total_score
      
    
    """
    This method calculates the score of point (x,y).
    For each point (x,y) we calculate vertical, horizontal, and both diagonal line score.
    """
    def point_score_calculator(self,game_state, x, y, player, opponent):
        score = 0

         # Calculates horizontal line score
        score += self.line_score_calculator(game_state=game_state,x=x,y=y,row_increament_direction=0,col_increm_direction=-1,first_row_cond=None,second_row_cond=None,first_col_cond=-1,second_col_cond=self.cols,player=player,opponent=opponent)

        # Calculates vertical line score
        score += self.line_score_calculator(game_state=game_state,x=x,y=y,row_increament_direction=-1,col_increm_direction=0,first_row_cond=-1,second_row_cond=self.rows,first_col_cond=None,second_col_cond=None,player=player,opponent=opponent)

         # Calculates diagonal y= -ax+b score
        score += self.line_score_calculator(game_state=game_state,x=x,y=y,row_increament_direction=-1,col_increm_direction=-1,first_row_cond=-1,second_row_cond=self.rows,first_col_cond=-1,second_col_cond=self.cols,player=player,opponent=opponent)

        # Calculates diagonal y=ax+b score
        score += self.line_score_calculator(game_state=game_state,x=x,y=y,row_increament_direction=-1,col_increm_direction=1,first_row_cond=-1,second_row_cond=self.rows,first_col_cond=self.cols,second_col_cond=-1,player=player,opponent=opponent)
        return score

    """
    This method searches horizontal, vertical, and diagonal lines and determines the score of (x,y).
    """
    def line_score_calculator(self,game_state,x,y,row_increament_direction,col_increm_direction,first_row_cond,second_row_cond,first_col_cond,second_col_cond,player,opponent):
        total_score = 0
        curr_in_line = 0
        values_in_a_row = 0
        values_in_a_row_previous = 0

        row = x + row_increament_direction
        column = y + col_increm_direction
        is_first = True
        while row != first_row_cond and column != first_col_cond and game_state[row][column] != 0:
            if is_first:
                curr_in_line = game_state[row][column]
                is_first = False
            if curr_in_line == game_state[row][column]:
                values_in_a_row += 1
            else:
                break
            row += row_increament_direction
            column += col_increm_direction

        row = x - row_increament_direction
        column = y - col_increm_direction
        is_first = True
        while row != second_row_cond and column != second_col_cond and game_state[row][column] != 0:
            if is_first:
                is_first = False

                if curr_in_line != game_state[row][column]:
                    if values_in_a_row == 3 and curr_in_line == player:
                        total_score += 1
                    elif values_in_a_row == 3 and curr_in_line == opponent:
                        total_score -= 1
                else:
                    values_in_a_row_previous = values_in_a_row

                values_in_a_row = 0
                curr_in_line = game_state[row][column]

            if curr_in_line == game_state[row][column]:
                values_in_a_row += 1
            else:
                break
            row -= row_increament_direction
            column -= col_increm_direction

        if values_in_a_row + values_in_a_row_previous >= 3 and curr_in_line == player:
            total_score += 1
        elif values_in_a_row + values_in_a_row_previous >= 3 and curr_in_line == opponent:
            total_score -= 1

        return total_score

    """
    This method runs the minmax method and returns the move to be selected. And also it checks if any immediate wins or loose is possible at given game state.
    """
    def optimal_column(self,game_state, player, opponent):
        for i in range(0, self.cols):
            # If moves cannot be made on i column, skip it
            if game_state[0][i] != 0:
                continue

            currentMove = [0, i]

            for j in range(0, self.rows - 1):
                if game_state[j + 1][i] != 0:
                    game_state[j][i] = player
                    currentMove[0] = j
                    break
                elif j == self.rows - 2:
                    game_state[j+1][i] = player
                    currentMove[0] = j+1

            winner = self.get_game_status(game_state)
            game_state[currentMove[0]][currentMove[1]] = 0

            if winner == self.CPU:
                return currentMove[1]

        for i in range(0, self.cols):
            # If moves cannot be made on i column, skip it
            if game_state[0][i] != 0:
                continue

            currentMove = [0, i]

            for j in range(0, self.rows - 1):
                if game_state[j + 1][i] != 0:
                    game_state[j][i] = opponent
                    currentMove[0] = j
                    break
                elif j == self.rows - 2:
                    game_state[j+1][i] = opponent
                    currentMove[0] = j+1

            winner = self.get_game_status(game_state)
            game_state[currentMove[0]][currentMove[1]] = 0

            if winner == self.HUMAN_PLAYER:
                return currentMove[1]

        move, score = self.minimax(game_state, self.search_depth, player, opponent)
        return move[1]
    
    """
    This method returns the game status. If this method returns 0, that means the no one won.
    """
    def get_game_status(self,game_state):
        if self.get_horizontal_status(game_state) != 0:
            return self.get_horizontal_status(game_state)
        
        if self.get_vertical_status(game_state) != 0:
            return self.get_vertical_status(game_state)
        
        if self.get_diagonal_status(game_state) != 0:
            return self.get_diagonal_status(game_state)
        
        # 0 means no one won!
        return 0
    
    def get_horizontal_status(self,game_state):
        CPU_connected_4 = 0
        YOU_connected_4 = 0
        curr = 0
        curr_count = 0

        for i in range(0, self.rows):
            for j in range(0, self.cols):
                if curr_count == 0:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count += 1
                elif curr_count == 4:
                    if curr == self.CPU:
                        CPU_connected_4 += 1
                    else:
                        YOU_connected_4 += 1
                    curr_count = 0
                    break
                elif game_state[i][j] != curr:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count = 1
                    else:
                        curr = 0
                        curr_count = 0
                else:
                    curr_count += 1

            if curr_count == 4:
                if curr == self.CPU:
                    CPU_connected_4 += 1
                else:
                    YOU_connected_4 += 1
            curr = 0
            curr_count = 0

        
        if YOU_connected_4 > 0:
            return self.HUMAN_PLAYER
        elif CPU_connected_4 > 0:
            return self.CPU
        else:
            return 0
    
    def get_horizontal_status(self,game_state):
        CPU_connected_4 = 0
        YOU_connected_4 = 0
        curr = 0
        curr_count = 0

        for i in range(0, self.rows):
            for j in range(0, self.cols):
                if curr_count == 0:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count += 1
                elif curr_count == 4:
                    if curr == self.CPU:
                        CPU_connected_4 += 1
                    else:
                        YOU_connected_4 += 1
                    curr_count = 0
                    break
                elif game_state[i][j] != curr:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count = 1
                    else:
                        curr = 0
                        curr_count = 0
                else:
                    curr_count += 1

            if curr_count == 4:
                if curr == self.CPU:
                    CPU_connected_4 += 1
                else:
                    YOU_connected_4 += 1
            curr = 0
            curr_count = 0

        
        if YOU_connected_4 > 0:
            return self.HUMAN_PLAYER
        elif CPU_connected_4 > 0:
            return self.CPU
        else:
            return 0
    
    def get_horizontal_status(self,game_state):
        CPU_connected_4 = 0
        YOU_connected_4 = 0
        curr = 0
        curr_count = 0

        for j in range(0, self.cols):
            for i in range(0, self.rows):
                if curr_count == 0:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count += 1
                elif curr_count == 4:
                    if curr == self.CPU:
                        CPU_connected_4 += 1
                    else:
                        YOU_connected_4 += 1
                    curr_count = 0
                    break
                elif game_state[i][j] != curr:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count = 1
                    else:
                        curr = 0
                        curr_count = 0
                else:
                    curr_count += 1

            if curr_count == 4:
                if curr == self.CPU:
                    CPU_connected_4 += 1
                else:
                    YOU_connected_4 += 1
            curr = 0
            curr_count = 0

        
        if YOU_connected_4 > 0:
            return self.HUMAN_PLAYER
        elif CPU_connected_4 > 0:
            return self.CPU
        else:
            return 0
    
    def get_vertical_status(self,game_state):
        CPU_connected_4 = 0
        YOU_connected_4 = 0
        curr = 0
        curr_count = 0

        for j in range(0, self.cols):
            for i in range(0, self.rows):
                if curr_count == 0:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count += 1
                elif curr_count == 4:
                    if curr == self.CPU:
                        CPU_connected_4 += 1
                    else:
                        YOU_connected_4 += 1
                    curr_count = 0
                    break
                elif game_state[i][j] != curr:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count = 1
                    else:
                        curr = 0
                        curr_count = 0
                else:
                    curr_count += 1

            if curr_count == 4:
                if curr == self.CPU:
                    CPU_connected_4 += 1
                else:
                    YOU_connected_4 += 1
            curr = 0
            curr_count = 0

        
        if YOU_connected_4 > 0:
            return self.HUMAN_PLAYER
        elif CPU_connected_4 > 0:
            return self.CPU
        else:
            return 0
    
    def get_diagonal_status(self,game_state):
        CPU_connected_4 = 0
        YOU_connected_4 = 0
        curr = 0
        curr_count = 0

        # Check diagonal wins
        np_matrix = np.array(game_state)
        diags = [np_matrix[::-1,:].diagonal(i) for i in range(-np_matrix.shape[0]+1,np_matrix.shape[1])]
        diags.extend(np_matrix.diagonal(i) for i in range(np_matrix.shape[1]-1,-np_matrix.shape[0],-1))
        diags_list = [n.tolist() for n in diags]

        for i in range(0, len(diags_list)):
            if len(diags_list[i]) >= 4:
                for j in range(0, len(diags_list[i])):
                    if curr_count == 0:
                        if diags_list[i][j] != 0:
                            curr = diags_list[i][j]
                            curr_count += 1
                    elif curr_count == 4:
                        if curr == self.CPU:
                            CPU_connected_4 += 1
                        else:
                            YOU_connected_4 += 1
                        curr_count = 0
                        break
                    elif diags_list[i][j] != curr:
                        if diags_list[i][j] != 0:
                            curr = diags_list[i][j]
                            curr_count = 1
                        else:
                            curr = 0
                            curr_count = 0
                    else:
                        curr_count += 1

                if curr_count == 4:
                    if curr == self.CPU:
                        CPU_connected_4 += 1
                    else:
                        YOU_connected_4 += 1
                curr = 0
                curr_count = 0


        
        if YOU_connected_4 > 0:
            return self.HUMAN_PLAYER
        elif CPU_connected_4 > 0:
            return self.CPU
        else:
            return 0


# MinMax with Alpha-beta pruning

In [4]:
class MinMaxHandlerAlphaBetaPruning:
    def __init__(self,cols = 7, rows = 6, search_depth = 4, CPU = -1, HUMAN_PLAYER = 1):
        self.cols = cols
        self.rows = rows
        self.search_depth = search_depth
        self.CPU = CPU
        self.HUMAN_PLAYER = HUMAN_PLAYER
    
    def calculate_number_of_possible_moves(self,game_state):
        possible_moves = self.cols
        for i in range(0, self.cols):
            if game_state[0][i] != 0:
                possible_moves -= 1
        return possible_moves

    def minimax(self,game_state, depth, player, opponent, alpha = float("-inf"), beta = float("inf")):
        possible_moves = self.calculate_number_of_possible_moves(game_state)
        
        # Evaluate Socre if there's possible moves or the depth is 0 and also return None for best_column
        if depth == 0 or possible_moves == 0:
            score = self.calculate_score(game_state, player, opponent)
            return None, score

        highest_score = None
        optimal_column = None

        for i in range(0, self.cols):
            # Skip this col if col is full
            if game_state[0][i] != 0:
                continue

            currentMove = [0, i]

            for j in range(0, self.rows - 1):
                if game_state[j + 1][i] != 0:
                    game_state[j][i] = player
                    currentMove[0] = j
                    break
                elif j == self.rows - 2:
                    game_state[j+1][i] = player
                    currentMove[0] = j+1

            # Recursive minimax call, with reduced depth
            move, score = self.minimax(game_state, depth - 1, opponent, player)

            game_state[currentMove[0]][currentMove[1]] = 0

            if player == self.CPU:
                if highest_score == None or score > highest_score:
                    highest_score = score
                    optimal_column = currentMove
                
                beta = min(beta, highest_score)

                if beta <= alpha:
                    break
            else:
                if highest_score == None or score < highest_score:
                    highest_score = score
                    optimal_column = currentMove
                
                alpha = max(alpha, highest_score)

                if beta <= alpha:
                    break

        return optimal_column, highest_score

    """
    This method calculates the score of each game state(board). 
    Return +inf if the you win and return -inf if CPU wins.
    """
    def calculate_score(self,game_state, player, opponent):
        # Return infinity if a self.YOU has won in the given board
        total_score = self.get_game_status(game_state)

        if total_score == player:
            return float("inf")
        elif total_score == opponent:
            return float("-inf")
        else:
            total_score = 0

        for i in range(0, self.rows):
            for j in range(0, self.cols):
                if game_state[i][j] == 0:
                    total_score += self.point_score_calculator(game_state, i, j, player, opponent)

        return total_score
      
    
    """
    This method calculates the score of point (x,y).
    For each point (x,y) we calculate vertical, horizontal, and both diagonal line score.
    """
    def point_score_calculator(self,game_state, x, y, player, opponent):
        score = 0

         # Calculates horizontal line score
        score += self.line_score_calculator(game_state=game_state,x=x,y=y,row_increament_direction=0,col_increm_direction=-1,first_row_cond=None,second_row_cond=None,first_col_cond=-1,second_col_cond=self.cols,player=player,opponent=opponent)

        # Calculates vertical line score
        score += self.line_score_calculator(game_state=game_state,x=x,y=y,row_increament_direction=-1,col_increm_direction=0,first_row_cond=-1,second_row_cond=self.rows,first_col_cond=None,second_col_cond=None,player=player,opponent=opponent)

         # Calculates diagonal y= -ax+b score
        score += self.line_score_calculator(game_state=game_state,x=x,y=y,row_increament_direction=-1,col_increm_direction=-1,first_row_cond=-1,second_row_cond=self.rows,first_col_cond=-1,second_col_cond=self.cols,player=player,opponent=opponent)

        # Calculates diagonal y=ax+b score
        score += self.line_score_calculator(game_state=game_state,x=x,y=y,row_increament_direction=-1,col_increm_direction=1,first_row_cond=-1,second_row_cond=self.rows,first_col_cond=self.cols,second_col_cond=-1,player=player,opponent=opponent)
        return score

    """
    This method searches horizontal, vertical, and diagonal lines and determines the score of (x,y).
    """
    def line_score_calculator(self,game_state,x,y,row_increament_direction,col_increm_direction,first_row_cond,second_row_cond,first_col_cond,second_col_cond,player,opponent):
        total_score = 0
        curr_in_line = 0
        values_in_a_row = 0
        values_in_a_row_previous = 0

        row = x + row_increament_direction
        column = y + col_increm_direction
        is_first = True
        while row != first_row_cond and column != first_col_cond and game_state[row][column] != 0:
            if is_first:
                curr_in_line = game_state[row][column]
                is_first = False
            if curr_in_line == game_state[row][column]:
                values_in_a_row += 1
            else:
                break
            row += row_increament_direction
            column += col_increm_direction

        row = x - row_increament_direction
        column = y - col_increm_direction
        is_first = True
        while row != second_row_cond and column != second_col_cond and game_state[row][column] != 0:
            if is_first:
                is_first = False

                if curr_in_line != game_state[row][column]:
                    if values_in_a_row == 3 and curr_in_line == player:
                        total_score += 1
                    elif values_in_a_row == 3 and curr_in_line == opponent:
                        total_score -= 1
                else:
                    values_in_a_row_previous = values_in_a_row

                values_in_a_row = 0
                curr_in_line = game_state[row][column]

            if curr_in_line == game_state[row][column]:
                values_in_a_row += 1
            else:
                break
            row -= row_increament_direction
            column -= col_increm_direction

        if values_in_a_row + values_in_a_row_previous >= 3 and curr_in_line == player:
            total_score += 1
        elif values_in_a_row + values_in_a_row_previous >= 3 and curr_in_line == opponent:
            total_score -= 1

        return total_score

    """
    This method runs the minmax method and returns the move to be selected. And also it checks if any immediate wins or loose is possible at given game state.
    """
    def optimal_column(self,game_state, player, opponent):
        for i in range(0, self.cols):
            # If moves cannot be made on i column, skip it
            if game_state[0][i] != 0:
                continue

            currentMove = [0, i]

            for j in range(0, self.rows - 1):
                if game_state[j + 1][i] != 0:
                    game_state[j][i] = player
                    currentMove[0] = j
                    break
                elif j == self.rows - 2:
                    game_state[j+1][i] = player
                    currentMove[0] = j+1

            winner = self.get_game_status(game_state)
            game_state[currentMove[0]][currentMove[1]] = 0

            if winner == self.CPU:
                return currentMove[1]

        for i in range(0, self.cols):
            # If moves cannot be made on i column, skip it
            if game_state[0][i] != 0:
                continue

            currentMove = [0, i]

            for j in range(0, self.rows - 1):
                if game_state[j + 1][i] != 0:
                    game_state[j][i] = opponent
                    currentMove[0] = j
                    break
                elif j == self.rows - 2:
                    game_state[j+1][i] = opponent
                    currentMove[0] = j+1

            winner = self.get_game_status(game_state)
            game_state[currentMove[0]][currentMove[1]] = 0

            if winner == self.HUMAN_PLAYER:
                return currentMove[1]

        move, score = self.minimax(game_state, self.search_depth, player, opponent)
        return move[1]
    
    """
    This method returns the game status. If this method returns 0, that means the no one won.
    """
    def get_game_status(self,game_state):
        if self.get_horizontal_status(game_state) != 0:
            return self.get_horizontal_status(game_state)
        
        if self.get_vertical_status(game_state) != 0:
            return self.get_vertical_status(game_state)
        
        if self.get_diagonal_status(game_state) != 0:
            return self.get_diagonal_status(game_state)
        
        # 0 means no one won!
        return 0
    
    def get_horizontal_status(self,game_state):
        CPU_connected_4 = 0
        YOU_connected_4 = 0
        curr = 0
        curr_count = 0

        for i in range(0, self.rows):
            for j in range(0, self.cols):
                if curr_count == 0:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count += 1
                elif curr_count == 4:
                    if curr == self.CPU:
                        CPU_connected_4 += 1
                    else:
                        YOU_connected_4 += 1
                    curr_count = 0
                    break
                elif game_state[i][j] != curr:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count = 1
                    else:
                        curr = 0
                        curr_count = 0
                else:
                    curr_count += 1

            if curr_count == 4:
                if curr == self.CPU:
                    CPU_connected_4 += 1
                else:
                    YOU_connected_4 += 1
            curr = 0
            curr_count = 0

        
        if YOU_connected_4 > 0:
            return self.HUMAN_PLAYER
        elif CPU_connected_4 > 0:
            return self.CPU
        else:
            return 0
    
    def get_horizontal_status(self,game_state):
        CPU_connected_4 = 0
        YOU_connected_4 = 0
        curr = 0
        curr_count = 0

        for i in range(0, self.rows):
            for j in range(0, self.cols):
                if curr_count == 0:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count += 1
                elif curr_count == 4:
                    if curr == self.CPU:
                        CPU_connected_4 += 1
                    else:
                        YOU_connected_4 += 1
                    curr_count = 0
                    break
                elif game_state[i][j] != curr:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count = 1
                    else:
                        curr = 0
                        curr_count = 0
                else:
                    curr_count += 1

            if curr_count == 4:
                if curr == self.CPU:
                    CPU_connected_4 += 1
                else:
                    YOU_connected_4 += 1
            curr = 0
            curr_count = 0

        
        if YOU_connected_4 > 0:
            return self.HUMAN_PLAYER
        elif CPU_connected_4 > 0:
            return self.CPU
        else:
            return 0
    
    def get_horizontal_status(self,game_state):
        CPU_connected_4 = 0
        YOU_connected_4 = 0
        curr = 0
        curr_count = 0

        for j in range(0, self.cols):
            for i in range(0, self.rows):
                if curr_count == 0:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count += 1
                elif curr_count == 4:
                    if curr == self.CPU:
                        CPU_connected_4 += 1
                    else:
                        YOU_connected_4 += 1
                    curr_count = 0
                    break
                elif game_state[i][j] != curr:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count = 1
                    else:
                        curr = 0
                        curr_count = 0
                else:
                    curr_count += 1

            if curr_count == 4:
                if curr == self.CPU:
                    CPU_connected_4 += 1
                else:
                    YOU_connected_4 += 1
            curr = 0
            curr_count = 0

        
        if YOU_connected_4 > 0:
            return self.HUMAN_PLAYER
        elif CPU_connected_4 > 0:
            return self.CPU
        else:
            return 0
    
    def get_vertical_status(self,game_state):
        CPU_connected_4 = 0
        YOU_connected_4 = 0
        curr = 0
        curr_count = 0

        for j in range(0, self.cols):
            for i in range(0, self.rows):
                if curr_count == 0:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count += 1
                elif curr_count == 4:
                    if curr == self.CPU:
                        CPU_connected_4 += 1
                    else:
                        YOU_connected_4 += 1
                    curr_count = 0
                    break
                elif game_state[i][j] != curr:
                    if game_state[i][j] != 0:
                        curr = game_state[i][j]
                        curr_count = 1
                    else:
                        curr = 0
                        curr_count = 0
                else:
                    curr_count += 1

            if curr_count == 4:
                if curr == self.CPU:
                    CPU_connected_4 += 1
                else:
                    YOU_connected_4 += 1
            curr = 0
            curr_count = 0

        
        if YOU_connected_4 > 0:
            return self.HUMAN_PLAYER
        elif CPU_connected_4 > 0:
            return self.CPU
        else:
            return 0
    
    def get_diagonal_status(self,game_state):
        CPU_connected_4 = 0
        YOU_connected_4 = 0
        curr = 0
        curr_count = 0

        # Check diagonal wins
        np_matrix = np.array(game_state)
        diags = [np_matrix[::-1,:].diagonal(i) for i in range(-np_matrix.shape[0]+1,np_matrix.shape[1])]
        diags.extend(np_matrix.diagonal(i) for i in range(np_matrix.shape[1]-1,-np_matrix.shape[0],-1))
        diags_list = [n.tolist() for n in diags]

        for i in range(0, len(diags_list)):
            if len(diags_list[i]) >= 4:
                for j in range(0, len(diags_list[i])):
                    if curr_count == 0:
                        if diags_list[i][j] != 0:
                            curr = diags_list[i][j]
                            curr_count += 1
                    elif curr_count == 4:
                        if curr == self.CPU:
                            CPU_connected_4 += 1
                        else:
                            YOU_connected_4 += 1
                        curr_count = 0
                        break
                    elif diags_list[i][j] != curr:
                        if diags_list[i][j] != 0:
                            curr = diags_list[i][j]
                            curr_count = 1
                        else:
                            curr = 0
                            curr_count = 0
                    else:
                        curr_count += 1

                if curr_count == 4:
                    if curr == self.CPU:
                        CPU_connected_4 += 1
                    else:
                        YOU_connected_4 += 1
                curr = 0
                curr_count = 0


        
        if YOU_connected_4 > 0:
            return self.HUMAN_PLAYER
        elif CPU_connected_4 > 0:
            return self.CPU
        else:
            return 0

# Game Logic

In [5]:
class ConnectSin:
    YOU = 1
    CPU = -1
    EMPTY = 0
    DRAW = 0
    __CONNECT_NUMBER = 4
    board = None

    def __init__(self, board_size=(6, 7), silent=False):
        """
        The main class for the connect4 game

        Inputs
        ----------
        board_size : a tuple representing the board size in format: (rows, columns)
        silent     : whether the game prints outputs or not
        """
        assert len(board_size) == 2, "board size should be a 1*2 tuple"
        assert board_size[0] > 4 and board_size[1] > 4, "board size should be at least 5*5"

        self.columns = board_size[1] 
        self.rows = board_size[0]
        self.silent = silent
        self.board_size = self.rows * self.columns
    
    
    def run(self, starter=1):
        """
        runs the game!

        Inputs
        ----------
        starter : either -1,1 or None. -1 if cpu starts the game, 1 if you start the game. None if you want the starter
            to be assigned randomly 

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that the game has drawn
        """
        if (not starter):
            starter = self.__get_random_starter()
        assert starter in [self.YOU, self.CPU], "starter value can only be 1,-1 or None"
        
        self.__init_board()

        
        turns_played = 0
        current_player = starter

        while(turns_played < self.board_size):
            
            if (current_player == self.YOU):
                self.__print_board()
                player_input = self.get_your_input()
            elif (current_player == self.CPU):
                player_input = self.__get_cpu_input()
                if player_input == None:
                    break
                player_input += 1
            else:
                raise Exception("A problem has happend! contact no one, there is no fix!")
            if (not self.register_input(player_input, current_player)):
                self.__print("this move is invalid!")
                continue
            current_player = self.__change_turn(current_player)
            potential_winner = self.check_for_winners()
            turns_played += 1
            if (potential_winner != 0):
                self.__print_board()
                self.__print_winner_message(potential_winner)
                return potential_winner
        self.__print_board()
        self.__print("The game has ended in a draw!")
        return self.DRAW

    def get_your_input(self):
        """
        gets your input

        Output
        ----------
        (int) an integer between 1 and column count. the column to put a piece in
        """
        
        if ALPHA_BETA_PRUNING == False:
            minmax_handler = MinMaxHandler(self.columns,self.rows,SEARCH_DEPTH,self.CPU,self.YOU)
        else:
            minmax_handler = MinMaxHandlerAlphaBetaPruning(self.columns,self.rows,SEARCH_DEPTH,self.CPU,self.YOU)
        return minmax_handler.optimal_column(self.board, self.CPU, self.YOU) + 1

    def check_for_winners(self):
        """
        checks if anyone has won in this position

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that nothing has happened
        """
        have_you_won = self.check_if_player_has_won(self.YOU)
        if have_you_won:
            return self.YOU
        has_cpu_won = self.check_if_player_has_won(self.CPU)
        if has_cpu_won:
            return self.CPU
        return self.EMPTY

    def check_if_player_has_won(self, player_id):
        """
        checks if player with player_id has won

        Inputs
        ----------
        player_id : the id for the player to check

        Output
        ----------
        (boolean) true if the player has won in this position
        """
        return (
            self.__has_player_won_diagonally(player_id)
            or self.__has_player_won_horizentally(player_id)
            or self.__has_player_won_vertically(player_id)
        )
    
    def is_move_valid(self, move):
        """
        checks if this move can be played

        Inputs
        ----------
        move : the column to place a piece in, in range [1, column count]

        Output
        ----------
        (boolean) true if the move can be played
        """
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return self.board[0][column_index] == 0
    
    def get_possible_moves(self):
        """
        returns a list of possible moves for the next move

        Output
        ----------
        (list) a list of numbers of columns that a piece can be placed in
        """
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid(move)):
                possible_moves.append(move)
        return possible_moves
    
    def register_input(self, player_input, current_player):
        """
        registers move to board, remember that this function changes the board

        Inputs
        ----------
        player_input : the column to place a piece in, in range [1, column count]
        current_player: ID of the current player, either self.YOU or self.CPU

        """
        if (not self.is_move_valid(player_input)):
            return False
        self.__drop_piece_in_column(player_input, current_player)
        return True

    def __init_board(self):
        self.board = []
        for i in range(self.rows):
            self.board.append([self.EMPTY] * self.columns)

    def __print(self, message: str):
        if not self.silent:
            print(message)

    def __has_player_won_horizentally(self, player_id):
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_vertically(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_diagonally(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __get_random_starter(self):
        players = [self.YOU, self.CPU]
        return players[int(random() * len(players))]
    
    def __drop_piece_in_column(self, move, current_player):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (self.board[i][column_index] == 0):
                last_empty_space = i
        self.board[last_empty_space][column_index] = current_player
        return True
        
    def __print_winner_message(self, winner):
        if (winner == self.YOU):
            self.__print("congrats! you have won!")
        else:
            self.__print("gg. CPU has won!")
    
    def __change_turn(self, turn):
        if (turn == self.YOU): 
            return self.CPU
        else:
            return self.YOU

    def __print_board(self):
        if (self.silent): return
        print("Y: you, C: CPU")
        for i in range(self.rows):
            for j in range(self.columns):
                house_char = "O"
                if (self.board[i][j] == self.YOU):
                    house_char = "Y"
                elif (self.board[i][j] == self.CPU):
                    house_char = "C"
                    
                print(f"{house_char}", end=" ")
            print()
    
    
    def __get_cpu_input(self):
        """
        This is where clean code goes to die.
        """
        bb = copy.deepcopy(self.board)
        pm = self.get_possible_moves()
        for m in pm:
            self.register_input(m, self.CPU)
            if (self.check_if_player_has_won(self.CPU)):
                self.board = bb
                return m
            self.board = copy.deepcopy(bb)
        if (self.is_move_valid((self.columns // 2) + 1)):
            c = 0
            cl = (self.columns // 2) + 1
            for x in range(self.rows):
                if (self.board[x][cl] == self.CPU):
                    c += 1
            if (random() < 0.65):
                return cl
        return pm[int(random() * len(pm))]


# Running one example

In [6]:
SEARCH_DEPTH = 4
ALPHA_BETA_PRUNING = False

board_sizes_to_check = [(6,7), 
                        (7,8), 
                        (7,10)]
game = ConnectSin(board_size=(6,7),silent=False)
start = time.time()
game.run()
end = time.time()
print("Time: %s seconds" % (end - start) , "\n")

Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
Y O O O C O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
Y O O O C O O 
Y O O O C O O 
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
Y O O O C O O 
Y O O O C O O 
Y O O O C O O 
Y: you, C: CPU
O O O O O O O 
O O O O C O O 
O O O O Y O O 
Y O O O C O O 
Y O O O C O O 
Y O O O C O O 
Y: you, C: CPU
O O O O O O O 
O O O O C O O 
Y O O O Y O O 
Y O O O C O O 
Y O O O C O O 
Y O O O C O O 
congrats! you have won!
Time: 1.8361177444458008 seconds 



In [45]:
def calculate_victory_chance(board_size,count = 200,search_depth = 4,pruning = False):
    SEARCH_DEPTH = search_depth
    ALPHA_BETA_PRUNING = pruning

    won_rounds = 0
    worst_execution_time = -1
    for i in range(count):
        game = ConnectSin(board_size=board_size,silent=True)
        start = time.time()
        # 1 is self.YOU
        if(game.run() == 1):
            won_rounds += 1
        end = time.time()

        exec_time = (end - start)
        worst_execution_time = max(worst_execution_time,exec_time)
        
    print("Execution time per round: %s seconds" % worst_execution_time)
    print("Victory Chance: ",str((won_rounds/count)*100)+"%")
    

# Without pruning

## search_depth = 1

In [34]:
calculate_victory_chance((6,7),search_depth=1,pruning=False)

Execution time per round: 2.0678925510406494 seconds
Victory Chance:  100.0%


In [35]:
calculate_victory_chance((7,8),search_depth=1,pruning=False)

Execution time per round: 4.100736951828003 seconds
Victory Chance:  100.0%


In [ ]:
calculate_victory_chance((7,10),search_depth=1,pruning=False)

Execution time per round: 11.980258703231812 seconds
Victory Chance:  100.0%


## search_depth = 3

In [27]:
calculate_victory_chance((6,7),search_depth=3,pruning=False)

Execution time per round: 1.984256479286193 seconds
Victory Chance:  100.0%


In [28]:
calculate_victory_chance((7,8),search_depth=3,pruning=False)

Execution time per round: 4.1001853692356 seconds
Victory Chance:  100.0%


In [ ]:
calculate_victory_chance((7,10),search_depth=3,pruning=False)

Execution time per round: 12.032996416091919 seconds
Victory Chance:  100.0%


## search_depth = 5


In [26]:
calculate_victory_chance((6,7),search_depth=5,pruning=False)

Execution time per round: 1.971895170211792 seconds
Victory Chance:  100.0%


In [29]:
calculate_victory_chance((7,8),search_depth=5,pruning=False)

Execution time per round: 4.21871853692356 seconds
Victory Chance:  100.0%


In [ ]:
calculate_victory_chance((7,10),search_depth=5,pruning=False)

Execution time per round: 12.065927743911743 seconds
Victory Chance:  100.0%


## search_depth = 7


In [ ]:
calculate_victory_chance((6,7),search_depth=7,pruning=False)

Execution time per round: 1.8072891235351562 seconds
Victory Chance:  100.0%


In [ ]:
calculate_victory_chance((7,8),search_depth=7,pruning=False)

Execution time per round: 3.966726779937744 seconds
Victory Chance:  100.0%


In [30]:
calculate_victory_chance((7,10),search_depth=7,pruning=False)

Execution time per round: 48.958349466323853 seconds
Victory Chance:  100.0%


# With alpha-beta pruning

## search_depth = 1

In [19]:
calculate_victory_chance((6,7),search_depth=1,pruning=True)

Execution time per round: 1.92844557762146 seconds
Victory Chance:  100.0%


In [ ]:
calculate_victory_chance((7,8),search_depth=1,pruning=True)


Execution time per round: 3.9984326362609863 seconds
Victory Chance:  100.0%


In [ ]:
calculate_victory_chance((7,10),search_depth=1,pruning=True)

Execution time per round: 11.965786933898926 seconds
Victory Chance:  100.0%


## search_depth = 3

In [ ]:
calculate_victory_chance((6,7),search_depth=3,pruning=True)


Execution time per round: 1.785003662109375 seconds
Victory Chance:  100.0%


In [ ]:
calculate_victory_chance((7,8),search_depth=3,pruning=True)


Execution time per round: 3.9781312942504883 seconds
Victory Chance:  100.0%


In [ ]:
calculate_victory_chance((7,10),search_depth=3,pruning=True)

Execution time per round: 11.90968370437622 seconds
Victory Chance:  100.0%


## search_depth = 5

In [ ]:
calculate_victory_chance((6,7),search_depth=5,pruning=True)


Execution time per round: 1.7722375392913818 seconds
Victory Chance:  100.0%


In [ ]:
calculate_victory_chance((7,8),search_depth=5,pruning=True)


Execution time per round: 4.034052133560181 seconds
Victory Chance:  100.0%


In [ ]:
calculate_victory_chance((7,10),search_depth=5,pruning=True)

Execution time per round: 11.929307222366333 seconds
Victory Chance:  100.0%


## search_depth = 7

In [ ]:
calculate_victory_chance((6,7),search_depth=7,pruning=True)

Execution time per round: 1.7761046886444092 seconds
Victory Chance:  100.0%


In [ ]:
calculate_victory_chance((7,8),search_depth=7,pruning=True)

Execution time per round: 3.9375979900360107 seconds
Victory Chance:  100.0%


In [31]:
calculate_victory_chance((7,10),search_depth=7,pruning=True)

Execution time per round: 17.925866603851318 seconds
Victory Chance:  100.0%


# Questions
## Question 1
A good heuristic is able to distinguish better game states from worse game states. That means if a given game state has more chance for our agent to win, has to have better heuristic value and if the given game state has less chance for our agent to to win, then the heuristic value for this state has to be lower.

The heurstic that I've chosen is better because the heurstic value for game states that are likely to win is high.
## Question 2
Increasing search depth could increase execution time, seen nodes, victory chance. Because higher search depth means the agent is able to see more moves in the future and make the best decision based on those game states.
## Question 3
The effectiveness of the alpha-beta pruning depends on the order in which children are visited.If children of a node are visited in the worst possible order, it may be that no pruning occurs.
- For max nodes, we want to visit the best child first so that time is not wasted in the rest of the children exploring worse scenarios.
- For min nodes, we want to visit the worst child first

The way we could improve order of children in this project is to sort them.

# Results

Here's how victory chance is calculated:

Victory Chance = #wins/#total games.

In this experiment, we run minmax algorithm with/without pruning 200 times.

In [2]:
import pandas as pd

## Without pruning

In [3]:
without_pruning = pd.DataFrame({
    'Board Size': ['(6,7)','(7,8)','(7,10)','(6,7)','(7,8)','(7,10)','(6,7)','(7,8)','(7,10)','(6,7)','(7,8)','(7,10)'],
    'Search Depth': [1,1,1,3,3,3,5,5,5,7,7,7],
    'Victory Chance': ['100%','100%','100%','100%','100%','100%','100%','100%','100%','100%','100%','100%'],
    'Exec. Time Per Round': ['2.0678925510406494 seconds','4.100736951828003 seconds','11.980258703231812 seconds',
                             '1.984256479286193 seconds','4.1001853692356 seconds','11.90968370437622 seconds',
                             '1.7722375392913818 seconds', '4.034052133560181 seconds','11.929307222366333 seconds',
                             '1.8072891235351562 seconds','3.9375979900360107 seconds','48.958349466323853 seconds']
})

without_pruning

,Board Size,Search Depth,Victory Chance,Exec. Time Per Round
0,"(6,7)",1,100%,2.0678925510406494 seconds
1,"(7,8)",1,100%,4.100736951828003 seconds
2,"(7,10)",1,100%,11.980258703231812 seconds
3,"(6,7)",3,100%,1.984256479286193 seconds
4,"(7,8)",3,100%,4.1001853692356 seconds
5,"(7,10)",3,100%,11.90968370437622 seconds
6,"(6,7)",5,100%,1.7722375392913818 seconds
7,"(7,8)",5,100%,4.034052133560181 seconds
8,"(7,10)",5,100%,11.929307222366333 seconds
9,"(6,7)",7,100%,1.8072891235351562 seconds


## With Alpha-Beta Pruning

In [4]:
alpha_beta_pruning = pd.DataFrame({
    'Board Size': ['(6,7)','(7,8)','(7,10)','(6,7)','(7,8)','(7,10)','(6,7)','(7,8)','(7,10)','(6,7)','(7,8)','(7,10)'],
    'Search Depth': [1,1,1,3,3,3,5,5,5,7,7,7],
    'Victory Chance': ['100%','100%','100%','100%','100%','100%','100%','100%','100%','100%','100%','100%'],
    'Exec. Time Per Round': ['1.92844557762146 seconds','3.9984326362609863 seconds','11.965786933898926 seconds',
                             '1.785003662109375 seconds','3.9781312942504883 seconds','12.032996416091919 seconds',
                             '1.971895170211792 seconds', '4.21871853692356 seconds','12.065927743911743 seconds',
                             '1.7761046886444092 seconds','3.966726779937744 seconds','17.925866603851318 seconds']
})

alpha_beta_pruning

,Board Size,Search Depth,Victory Chance,Exec. Time Per Round
0,"(6,7)",1,100%,1.92844557762146 seconds
1,"(7,8)",1,100%,3.9984326362609863 seconds
2,"(7,10)",1,100%,11.965786933898926 seconds
3,"(6,7)",3,100%,1.785003662109375 seconds
4,"(7,8)",3,100%,3.9781312942504883 seconds
5,"(7,10)",3,100%,12.032996416091919 seconds
6,"(6,7)",5,100%,1.971895170211792 seconds
7,"(7,8)",5,100%,4.21871853692356 seconds
8,"(7,10)",5,100%,12.065927743911743 seconds
9,"(6,7)",7,100%,1.7761046886444092 seconds
